In [ ]:
import json
from requests_oauthlib import OAuth2Session
from requests.auth import HTTPBasicAuth
import os
from datetime import datetime, timedelta
from dotenv import load_dotenv

load_dotenv()


True

In [37]:
print(os.getenv("CLIENT_ID"))
print(os.getenv("CLIENT_SECRET"))
print(os.getenv("AUTH_TOKEN_URL"))
print(os.getenv("AUTH_URL"))
print(os.getenv("REDIRECT_URI"))

58faffc8452d44a2aa81e039544b8c0d
4e8ee3677d2a4f2c8975088c89c9ecec
None
https://accounts.spotify.com/api/token
https://localhost:8001/callback


In [42]:
class SpotifyConn:
    def __init__(
        self, 
        client_id=os.getenv("CLIENT_ID"), 
        client_secret=os.getenv("CLIENT_SECRET"), 
        token_url=os.getenv("AUTH_TOKEN_URL"), 
        auth_url=os.getenv("AUTH_URL"), 
        redirect_uri=os.getenv("REDIRECT_URI"), 
        data_path="my_data", 
        picklefile="ids_added_in_db", 
        username=os.getenv("CLIENT_USERNAME"), 
        scope=None):
        if scope is None:
            scope = ['user-read-recently-played']
        self.client_id = client_id
        self.client_secret = client_secret
        self.token_url = token_url
        self.auth_url = auth_url
        self.redirect_uri = redirect_uri
        self.data_path = data_path
        self.picklefile = picklefile
        self.username = username
        self.cache_file = f".cache-{self.username}"
        self.scope = scope
        self.token = self._get_token_info()
    
    @property
    def spotify(self):
        return OAuth2Session(
            self.client_id,
            scope=self.scope, 
            redirect_uri=self.redirect_uri
        )
    
    def _get_token_info(self):
        if not os.path.exists(self.cache_file):
            #No cached value, get and cache
            return "no cached token go figure"
        #Reading cache
        with open(self.cache_file, "r") as infile:
            cached_token_data = json.load(infile)

        return "expired token" if int(datetime.now().timestamp()) > cached_token_data['expires_at'] else "valid access token"
    
    def get_token(self):
        # Redirect user to Spotify for authorization
        authorization_url, state = self.spotify.authorization_url(self.auth_url)
        print('Please go here and authorize: ', authorization_url)
        redirect_response = input('Paste the full redirect URL here: ')
        auth = HTTPBasicAuth(self.client_id, self.client_secret)
        # Fetch the access token
        token = self.spotify.fetch_token(self.token_url, auth=auth,
                authorization_response=redirect_response)
        print(token)
        with open(self.cache_file, "w") as outfile:
                json.dump(token, outfile)




In [43]:
conn = SpotifyConn()

KeyError: 'expiration_date'

In [41]:
conn.cache_file

'.cache-None'

In [8]:
# Credentials you get from registering a new application
cache_file = ".cache-juliansunn"
client_id = '58faffc8452d44a2aa81e039544b8c0d'
client_secret = '4e8ee3677d2a4f2c8975088c89c9ecec'
redirect_uri = 'https://localhost:8888/callback/'
# OAuth endpoints given in the Spotify API documentation
# https://developer.spotify.com/documentation/general/guides/authorization/code-flow/
authorization_base_url = "https://accounts.spotify.com/authorize"
token_url = "https://accounts.spotify.com/api/token"
# https://developer.spotify.com/documentation/general/guides/authorization/scopes/
scope = [
    "user-read-email",
    "playlist-read-collaborative"
]
spotify = OAuth2Session(client_id, scope=scope, redirect_uri=redirect_uri)
# Redirect user to Spotify for authorization
authorization_url, state = spotify.authorization_url(authorization_base_url)
print('Please go here and authorize: ', authorization_url)
# Get the authorization verifier code from the callback url



Please go here and authorize:  https://accounts.spotify.com/authorize?response_type=code&client_id=58faffc8452d44a2aa81e039544b8c0d&redirect_uri=https%3A%2F%2Flocalhost%3A8888%2Fcallback%2F&scope=user-read-email+playlist-read-collaborative&state=aSjnXY2hxvwSISEtouhTajCfJd5tW7


In [5]:

redirect_response = input('Paste the full redirect URL here: ')
auth = HTTPBasicAuth(client_id, client_secret)
# Fetch the access token
token = spotify.fetch_token(token_url, auth=auth,
        authorization_response=redirect_response)
print(token)
with open(cache_file, "w") as outfile:
        json.dump(token, outfile)
# Fetch a protected resource, i.e. user profile
r = spotify.get('https://api.spotify.com/v1/me')
print(r.content)

{'access_token': 'BQDvlXYQb1XLYC_J8fupt4uPcVuID-u1p4XY7hGyM9s9trUGJlLcL4Mfq5C1uzIIeycUBRf1N05fwLa9H3xSuJYhDL2UMrJtz-lF6ipKGb5DxriCzPZQUSGZZUqPNBkIDMzzOsHKbNN_hFBu4c9vHSQRBLjwX0ZYfwU3I0Nz9AsfotDF7GsImRgQNQ', 'token_type': 'Bearer', 'expires_in': 3600, 'refresh_token': 'AQBGlabACBcsAWYqVzVMLPi5O_RF2IIyCafo1TRYH1BsaubfuIBdDYxvZ3ey74coESa0ze0NEsj5m6k1O2wXC7Tepmy-JbPQczk7Hc_DRA8FKtFe0OU5_R0j3JfLY4l-Xus', 'scope': ['playlist-read-collaborative', 'user-read-email'], 'expires_at': 1664563683.9333124}
b'{\n  "display_name" : "Julian Sunn",\n  "email" : "juliansunn@yahoo.com",\n  "external_urls" : {\n    "spotify" : "https://open.spotify.com/user/1210837287"\n  },\n  "followers" : {\n    "href" : null,\n    "total" : 16\n  },\n  "href" : "https://api.spotify.com/v1/users/1210837287",\n  "id" : "1210837287",\n  "images" : [ {\n    "height" : null,\n    "url" : "https://scontent-ort2-2.xx.fbcdn.net/v/t31.18172-1/12356788_10102074074210518_5822717779330633814_o.jpg?stp=dst-jpg_p320x320&_nc_cat=103&

In [26]:
spotify.refresh_token(token_url, refresh_token=token['refresh_token'], client_id=client_id, client_secret=client_secret)


{'access_token': 'BQB0UUCssbOSG6L22DC2xCBt0xs_yEFr6EmIqryGOdZeGpStWdO6v44tiPQB-3Kh_D9D6QDBMk4angYhAtw3WPdbNUFmYWFQngkiDJ7uPE5XZ2bdP_daEGGhLJz-0JLEfDIyEnKaFkq9HUiLzTvjYZ2pZGAdYd8X9h2ShYmLcPufQFw1qbVLh4lv2w',
 'token_type': 'Bearer',
 'expires_in': 3600,
 'scope': ['playlist-read-collaborative', 'user-read-email'],
 'expires_at': 1664566598.0931602,
 'refresh_token': 'AQBGlabACBcsAWYqVzVMLPi5O_RF2IIyCafo1TRYH1BsaubfuIBdDYxvZ3ey74coESa0ze0NEsj5m6k1O2wXC7Tepmy-JbPQczk7Hc_DRA8FKtFe0OU5_R0j3JfLY4l-Xus'}

In [ ]:
df = hist_df[~hist_df['ID'].isin(conn.ids_in_db)]

In [ ]:
df

In [ ]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [ ]:
Track.objects.all()